# Adding the $8\pi T_{\mu\nu}$ Matter Source Terms to Einstein's equations of general relativity in the [BSSN](http://www2.yukawa.kyoto-u.ac.jp/~yuichiro.sekiguchi/3+1.pdf) formalism, in ***curvilinear*** coordinates, using a covariant reference metric approach: C code generation of the evolution equations' right-hand sides

### ***Citations***: Generic curvilinear coordinate reference metric approach matches that of [Ruchlin, Etienne, and Baumgarte (2018)](https://arxiv.org/abs/1712.07658), which is an extension of the spherical coordinate reference metric approach of [Baumgarte, Montero, Cordero-Carrión, and Müller (2012)](https://arxiv.org/abs/1211.6632), which builds upon the covariant "Lagrangian" BSSN formalism of [Brown (2009)](https://arxiv.org/abs/0902.3652). *See also citations within each article.*

In the [previous module](Tutorial-BSSNCurvilinear.ipynb), we generated SymPy expressions for all of the covariant BSSN equations in curvilinear coordinates, rescaling tensors to factor out coordinate singularities (and render them harmless). To wit:

\begin{align}
  \partial_t \bar{\gamma}_{i j} {} = {} & \left[\beta^k \partial_k \bar{\gamma}_{ij} + \partial_i \beta^k \bar{\gamma}_{kj} + \partial_j \beta^k \bar{\gamma}_{ik} \right] + \frac{2}{3} \bar{\gamma}_{i j} \left (\alpha \bar{A}_{k}^{k} - \bar{D}_{k} \beta^{k}\right ) - 2 \alpha \bar{A}_{i j} \; , \\
  \partial_t \bar{A}_{i j} {} = {} & \left[\beta^k \partial_k \bar{A}_{ij} + \partial_i \beta^k \bar{A}_{kj} + \partial_j \beta^k \bar{A}_{ik} \right] - \frac{2}{3} \bar{A}_{i j} \bar{D}_{k} \beta^{k} - 2 \alpha \bar{A}_{i k} {\bar{A}^{k}}_{j} + \alpha \bar{A}_{i j} K \nonumber \\
  & + e^{-4 \phi} \left \{-2 \alpha \bar{D}_{i} \bar{D}_{j} \phi + 4 \alpha \bar{D}_{i} \phi \bar{D}_{j} \phi  + 4 \bar{D}_{(i} \alpha \bar{D}_{j)} \phi - \bar{D}_{i} \bar{D}_{j} \alpha + \alpha \bar{R}_{i j} \right \}^{\text{TF}} \; , \\
  \partial_t \phi {} = {} & \left[\beta^k \partial_k \phi \right] + \frac{1}{6} \left (\bar{D}_{k} \beta^{k} - \alpha K \right ) \; , \\
  \partial_{t} K {} = {} & \left[\beta^k \partial_k K \right] + \frac{1}{3} \alpha K^{2} + \alpha \bar{A}_{i j} \bar{A}^{i j} - e^{-4 \phi} \left (\bar{D}_{i} \bar{D}^{i} \alpha + 2 \bar{D}^{i} \alpha \bar{D}_{i} \phi \right ) \; , \\
  \partial_t \bar{\Lambda}^{i} {} = {} & \left[\beta^k \partial_k \bar{\Lambda}^i - \partial_k \beta^i \bar{\Lambda}^k \right] + \bar{\gamma}^{j k} \hat{D}_{j} \hat{D}_{k} \beta^{i} + \frac{2}{3} \Delta^{i} \bar{D}_{j} \beta^{j} + \frac{1}{3} \bar{D}^{i} \bar{D}_{j} \beta^{j} \nonumber \\
  & - 2 \bar{A}^{i j} \left (\partial_{j} \alpha - 6 \partial_{j} \phi \right ) + 2 \alpha \bar{A}^{j k} \Delta_{j k}^{i}  -\frac{4}{3} \alpha \bar{\gamma}^{i j} \partial_{j} K \\
\partial_t \alpha &= \left[\beta^i \partial_i \alpha\right] - 2 \alpha K \\
  \partial_{t} \beta^{i} &= \left[\beta^j \partial_j \beta^i\right] + B^{i} \\
  \partial_{t} B^{i} &= \left[\beta^j \partial_j B^i\right] + \frac{3}{4} \partial_{0} \bar{\Lambda}^{i} - \eta B^{i}, \; .
\end{align}

where the terms moved from the left-hand side are enclosed in square braces. Note that $\partial_{0} \bar{\Lambda}^{i}$ in the right-hand side of the $\partial_{t} B^{i}$ equation is computed by adding $\beta^j \partial_j \bar{\Lambda}^i$ to the right-hand side expression given for $\partial_t \bar{\Lambda}^i$, so no explicit time dependence occurs in the right-hand sides of the BSSN evolution equations and the Method of Lines can be applied directly.

Notice that the shift advection operator $\beta^k \partial_k \left\{\bar{\gamma}_{i j},\bar{A}_{i j},\phi, K, \bar{\Lambda}^{i}, \alpha, \beta^i, B^i\right\}$ appears on the right-hand side of *every* expression. As the shift determines how the spatial coordinates $x^i$ move on the next 3D slice of our 4D manifold, we find that representing $\partial_k$ in these shift advection terms via an *upwinded* finite difference stencil results in far lower numerical errors. This trick is implemented below in all shift advection terms.

Of course the above BSSN evolution equations cannot appear as written above in our actual code, because (as discussed in the [previous module](Tutorial-BSSNCurvilinear.ipynb)) tensorial expressions can diverge at coordinate singularities. So the equations above will simply need to be rescaled according to the prescription (again, described in the [previous module](Tutorial-BSSNCurvilinear.ipynb)).

## Adding the matter source terms

Analyzing Eqs. 9 of [Baumgarte, Montero, Cordero-Carrión, and Müller](https://arxiv.org/pdf/1211.6632.pdf), we see that matter source terms $T_{\mu\nu}$ to Einstein's equations only adjust the right-hand sides of the $\partial_t \bar{A}_{ij}$, $\partial_t K$, and $\partial_t \bar{\Lambda}^i$ equations, as follows:


\begin{array}
\ \partial_t \bar{A}_{ij} &= [\text{as above}]\ {\color{blue}{-\ 8\pi \alpha e^{4\phi} \left(S_{ij}\right)^{\rm TF}}} \\
\partial_t K &= [\text{as above}]\ {\color{blue}{+\ 4\pi \alpha (\rho + S)}} \\
\partial_t \bar{\Lambda}^i &= [\text{as above}]\ {\color{blue}{-\ 16\pi \alpha \gamma^{ij} S_j}}, \\
\end{array}

where $\rho$, $S$, $S_i$, and $S_{ij}$ are related to $T^{\mu\nu}$ as follows (note that Greek indices are 4-dimensional and Latin indices are 3-dimensional):

\begin{array}
\ S_{ij} &= \gamma_{ik} \gamma_{jl} T^{kl} \\
S_{i} &= -\gamma_{ik} n_\nu T^{k\nu} \\
S &= \gamma^{ij} S_{ij} \\
\rho &= n_\mu n_\nu T^{\mu\nu}
\end{array}